# Task 1: Robot Navigation in a Grid Warehouse

This notebook implements and visualizes search algorithms (BFS and A*) for a robot navigating a 2D grid warehouse with obstacles.

## 1. Setup and Imports

Import necessary libraries: `numpy` for grid manipulation, `matplotlib` for visualization, `collections.deque` for BFS queue, and `heapq` for A* priority queue.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from collections import deque
import heapq
import time
from IPython.display import HTML, display

## 2. Define the Grid Environment

Define the warehouse grid layout. 
- `0`: Free space
- `1`: Obstacle ('X')
- `2`: Start ('S')
- `3`: Goal ('G')

Also, define the start and goal coordinates.

In [ ]:
grid = np.array([
    [2, 0, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 1, 3]
])

start_pos = tuple(np.argwhere(grid == 2)[0])
goal_pos = tuple(np.argwhere(grid == 3)[0])

rows, cols = grid.shape

## 3. Visualization Function

Create a function to visualize the grid, the path found, and the robot's movement step-by-step.

In [ ]:
def visualize_search(grid, path, explored_nodes=None, title='Robot Navigation'):
    fig, ax = plt.subplots(figsize=(6, 6))
    cmap = plt.cm.colors.ListedColormap(['white', 'black', 'blue', 'green', 'red', 'yellow'])
    # 0: Free, 1: Obstacle, 2: Start, 3: Goal, 4: Path, 5: Explored
    
    vis_grid = grid.copy().astype(float) # Use float for visualization codes
    
    # Mark explored nodes if provided
    if explored_nodes:
        for node in explored_nodes:
            if vis_grid[node] == 0: # Only mark free space as explored
                 vis_grid[node] = 5 # Yellow for explored

    # Mark the path
    if path:
        for step in path:
            if vis_grid[step] == 0 or vis_grid[step] == 5: # Mark free or explored space as path
                vis_grid[step] = 4 # Red for path
    
    # Ensure Start and Goal retain their colors
    vis_grid[start_pos] = 2 # Blue for Start
    vis_grid[goal_pos] = 3 # Green for Goal

    ax.imshow(vis_grid, cmap=cmap, interpolation='nearest', vmin=0, vmax=5)

    # Draw gridlines
    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which="minor", color="grey", linestyle='-', linewidth=0.5)
    ax.tick_params(which="minor", size=0)
    ax.set_xticks(np.arange(0, cols, 1))
    ax.set_yticks(np.arange(0, rows, 1))
    ax.set_xticklabels(np.arange(0, cols, 1))
    ax.set_yticklabels(np.arange(0, rows, 1))

    plt.title(title)
    
    # Create legend handles manually
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, fc="white", ec="black", label='Free Space'),
        plt.Rectangle((0, 0), 1, 1, fc="black", ec="black", label='Obstacle (X)'),
        plt.Rectangle((0, 0), 1, 1, fc="blue", ec="black", label='Start (S)'),
        plt.Rectangle((0, 0), 1, 1, fc="green", ec="black", label='Goal (G)'),
        plt.Rectangle((0, 0), 1, 1, fc="red", ec="black", label='Path'),
        plt.Rectangle((0, 0), 1, 1, fc="yellow", ec="black", label='Explored')
    ]
    ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.show()

def animate_search(grid, path, explored_frames, title='Robot Navigation Animation'):
    fig, ax = plt.subplots(figsize=(6, 6))
    cmap = plt.cm.colors.ListedColormap(['white', 'black', 'blue', 'green', 'red', 'yellow'])
    # 0: Free, 1: Obstacle, 2: Start, 3: Goal, 4: Path, 5: Explored

    vis_grid = grid.copy().astype(float)
    vis_grid[start_pos] = 2
    vis_grid[goal_pos] = 3

    im = ax.imshow(vis_grid, cmap=cmap, interpolation='nearest', vmin=0, vmax=5)

    # Draw gridlines
    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which="minor", color="grey", linestyle='-', linewidth=0.5)
    ax.tick_params(which="minor", size=0)
    ax.set_xticks(np.arange(0, cols, 1))
    ax.set_yticks(np.arange(0, rows, 1))
    ax.set_xticklabels(np.arange(0, cols, 1))
    ax.set_yticklabels(np.arange(0, rows, 1))
    plt.title(title)

    # Legend
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, fc="white", ec="black", label='Free Space'),
        plt.Rectangle((0, 0), 1, 1, fc="black", ec="black", label='Obstacle (X)'),
        plt.Rectangle((0, 0), 1, 1, fc="blue", ec="black", label='Start (S)'),
        plt.Rectangle((0, 0), 1, 1, fc="green", ec="black", label='Goal (G)'),
        plt.Rectangle((0, 0), 1, 1, fc="red", ec="black", label='Final Path'),
        plt.Rectangle((0, 0), 1, 1, fc="yellow", ec="black", label='Explored')
    ]
    ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')

    # Animation update function
    def update(frame):
        current_grid = vis_grid.copy()
        # Mark explored nodes up to the current frame
        for i in range(min(frame + 1, len(explored_frames))):
            node = explored_frames[i]
            if current_grid[node] == 0: # Only mark free space
                current_grid[node] = 5 # Yellow for explored
        
        # If exploration is done, show the final path
        if frame >= len(explored_frames) and path:
            path_frame_index = frame - len(explored_frames)
            # Mark explored nodes fully
            for node in explored_frames:
                 if current_grid[node] == 0 or current_grid[node] == 5:
                     current_grid[node] = 5
            # Mark path incrementally
            for i in range(min(path_frame_index + 1, len(path))):
                step = path[i]
                if current_grid[step] == 0 or current_grid[step] == 5: # Mark free or explored space as path
                    current_grid[step] = 4 # Red for path
        
        # Ensure Start and Goal retain their colors
        current_grid[start_pos] = 2
        current_grid[goal_pos] = 3
        
        im.set_data(current_grid)
        return [im]

    # Calculate total frames needed (exploration + path drawing)
    total_frames = len(explored_frames) + (len(path) if path else 0)
    
    # Create animation
    ani = animation.FuncAnimation(fig, update, frames=total_frames, interval=150, blit=True, repeat=False)
    plt.close(fig) # Prevent duplicate static plot
    return ani

## 4. Search Algorithms

Implement the Breadth-First Search (BFS) and A* search algorithms.

### 4.1 Helper Functions

`get_neighbors`: Returns valid neighboring cells (within bounds, not obstacles).
`reconstruct_path`: Traces back the path from goal to start using the `came_from` dictionary.
`heuristic`: Calculates the Manhattan distance for A*.

In [ ]:
def get_neighbors(node, grid):
    neighbors = []
    rows, cols = grid.shape
    r, c = node
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)] # Right, Left, Down, Up

    for dr, dc in moves:
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols and grid[nr, nc] != 1: # Check bounds and obstacles
            neighbors.append((nr, nc))
    return neighbors

def reconstruct_path(came_from, start, goal):
    current = goal
    path = []
    if goal not in came_from:
        return [] # Goal not reachable
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    return path[::-1] # Return reversed path

def heuristic(a, b):
    # Manhattan distance
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

### 4.2 Breadth-First Search (BFS)

BFS explores the grid layer by layer, guaranteeing the shortest path in terms of the number of steps for unweighted graphs.

In [ ]:
def bfs(grid, start, goal):
    queue = deque([(start, [start])]) # Store (node, path_to_node)
    visited = {start}
    came_from = {start: None}
    explored_nodes_order = [start] # Track exploration order for visualization
    
    while queue:
        current_node, path = queue.popleft()
        
        if current_node == goal:
            final_path = reconstruct_path(came_from, start, goal)
            return final_path, explored_nodes_order
            
        for neighbor in get_neighbors(current_node, grid):
            if neighbor not in visited:
                visited.add(neighbor)
                came_from[neighbor] = current_node
                explored_nodes_order.append(neighbor)
                new_path = list(path)
                new_path.append(neighbor)
                queue.append((neighbor, new_path))
                
    return None, explored_nodes_order # Path not found

### 4.3 A* Search

A* uses a heuristic (Manhattan distance) to prioritize nodes closer to the goal, often finding the optimal path more efficiently than BFS.

In [ ]:
def a_star(grid, start, goal):
    open_set = [(0, start)] # Priority queue: (f_score, node)
    heapq.heapify(open_set)
    
    came_from = {start: None}
    g_score = {node: float('inf') for node in np.ndindex(grid.shape)}
    g_score[start] = 0
    f_score = {node: float('inf') for node in np.ndindex(grid.shape)}
    f_score[start] = heuristic(start, goal)
    
    open_set_hash = {start} # To check membership efficiently
    explored_nodes_order = [] # Track exploration order

    while open_set:
        current_f, current_node = heapq.heappop(open_set)
        open_set_hash.remove(current_node)
        explored_nodes_order.append(current_node)

        if current_node == goal:
            path = reconstruct_path(came_from, start, goal)
            return path, explored_nodes_order

        for neighbor in get_neighbors(current_node, grid):
            # Assume cost between neighbors is 1
            tentative_g_score = g_score[current_node] + 1
            
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                if neighbor not in open_set_hash:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
                    open_set_hash.add(neighbor)
                    
    return None, explored_nodes_order # Path not found

## 5. Run Search and Visualize

Choose an algorithm ('BFS' or 'A*'), run the search, and visualize the results.

In [ ]:
def run_and_visualize(algorithm_name):
    print(f"Running {algorithm_name}...")
    start_time = time.time()
    
    if algorithm_name == 'BFS':
        path, explored = bfs(grid, start_pos, goal_pos)
    elif algorithm_name == 'A*':
        path, explored = a_star(grid, start_pos, goal_pos)
    else:
        print("Invalid algorithm choice.")
        return

    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.4f} seconds")

    if path:
        print(f"Path found with length: {len(path) - 1}")
        # Static visualization of the final path and explored nodes
        visualize_search(grid, path, explored, title=f'{algorithm_name} - Final Path and Explored Nodes')
        
        # Animation
        print("Generating animation...")
        ani = animate_search(grid, path, explored, title=f'{algorithm_name} - Search Animation')
        # Display the animation in Colab/Jupyter
        display(HTML(ani.to_jshtml()))
        # To save animation (optional):
        # ani.save(f'{algorithm_name}_animation.gif', writer='pillow', fps=5)
        # print(f"Animation saved as {algorithm_name}_animation.gif")
    else:
        print("Path not found.")
        # Visualize explored nodes even if path not found
        visualize_search(grid, None, explored, title=f'{algorithm_name} - Explored Nodes (No Path Found)')


### 5.1 Run BFS

In [ ]:
run_and_visualize('BFS')

### 5.2 Run A*

In [ ]:
run_and_visualize('A*')

## 6. Explanation

### Approach Used

1.  **Grid Representation**: A NumPy array represents the warehouse, with different integers encoding free space, obstacles, start, and goal.
2.  **Search Algorithms**: 
    *   **BFS**: Explores level by level using a queue. It guarantees the shortest path in terms of steps but can be inefficient in large spaces as it explores all reachable nodes at a given depth before moving deeper.
    *   **A***: Uses a priority queue guided by `f_score = g_score + h_score`. `g_score` is the cost from the start, and `h_score` (heuristic) estimates the cost to the goal (Manhattan distance used here). It prioritizes promising paths, often finding the optimal path faster than BFS.
3.  **Path Reconstruction**: A `came_from` dictionary tracks the predecessor of each node, allowing the path to be rebuilt once the goal is found.
4.  **Visualization**: `matplotlib` is used to display the grid, obstacles, start, goal, explored nodes, and the final path. `matplotlib.animation` creates a step-by-step visualization of the search process (exploration) followed by the robot tracing the final path.

### Pros and Cons

**BFS:**
*   **Pros**: Guarantees the shortest path (in terms of number of steps). Simple to implement. Complete (finds a path if one exists).
*   **Cons**: Can be slow and memory-intensive in large grids as it explores exhaustively. Doesn't consider the cost or heuristic estimate to the goal.

**A*:**
*   **Pros**: Optimal (guarantees the least-cost path if the heuristic is admissible, like Manhattan distance). Often much faster than BFS, especially in large grids, as it focuses exploration towards the goal. Complete.
*   **Cons**: Slightly more complex to implement than BFS. Performance depends on the quality of the heuristic.

### Assumptions Made

1.  **Uniform Cost**: The cost of moving between adjacent cells (up, down, left, right) is assumed to be 1.
2.  **Static Environment**: The grid (obstacles, start, goal) does not change during the search.
3.  **Valid Grid**: The grid contains exactly one start ('S') and one goal ('G').
4.  **Admissible Heuristic (A*)**: The Manhattan distance heuristic used is admissible (it never overestimates the actual cost to the goal) and consistent, ensuring A* finds the optimal path.